# A Walkthrough of the Code
- Information on how the program runs in the background is given here
- This is mostly for documentation. (The user doesn't have to be concerned with these details)

# Setup
- Code is stored in separate protds_v2.py file for cleaner Notebook use
- Start by assigning name of input file (stored in the data folder)
- The program refers to entries through column names, so any csv will work as long as the header names contain:  
    "ProteinID", "ModifiedLocationNum", and "ModifiedSequence"

<font color='red'>Some warnings may come up due to packages using older function names (or bad practice), but everything still works.   
    (Minor issue. Will fix later, if possible)</font> 

In [1]:
from protds_v2 import *

filename = 'Peptide_IndexByID_Version_test.csv'
data = pd.read_csv(os.path.join(os.pardir,"data",filename))

C:\Users\SUL\anaconda3\envs\snowflakes\lib\site-packages\networkx\readwrite\graphml.py:346: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.int, "int"), (np.int8, "int"),
C:\Users\SUL\anaconda3\envs\snowflakes\lib\site-packages\networkx\readwrite\gexf.py:220: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to r

In [2]:
moddata = data[data['ModifiedLocationNum'].notna()] #select rows with Modifications
moddata['ModifiedLocationNum'] = moddata['ModifiedLocationNum'].astype(int) #remove decimals in ModifiedLocationNum 
moddata 

C:\Users\SUL\AppData\Local\Temp/ipykernel_22684/300872404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moddata['ModifiedLocationNum'] = moddata['ModifiedLocationNum'].astype(int) #remove decimals in ModifiedLocationNum


,ProteinID,PeptideSequence,Peptide_Start,Peptide_End,Proteinlength,ProteinSequence,ModifiedSequence,ModifiedLocation,ModifiedAA,ModifiedLocationNum,ModifiedType,5min,20min,60min
6534,P00350,VAASKVLSGPQAQPAGDK,297,314,468,MSKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAEN...,VAASK[NHS-OH]VLSGPQAQPAGDK,K301,K,301,NHS-OH,10.097795,42.046806,32.038967
6535,P00350,IAAVAEDGEPCVTYIGADGAGHYVK,159,183,468,MSKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAEN...,IAAVAEDGEPC[Carbamidomethyl (C)]VTYIGADGAGHYVK,C169,C,169,Carbamidomethyl (C),199.737198,317.574097,215.253174
6536,P00363,AAIAAAQANPNAKIALISK,20,38,602,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...,AAIAAAQANPNAK[NHS-OH]IALISK,K32,K,32,NHS-OH,5.593114,4.614741,17.737122
6537,P00363,LPFICELAK,329,337,602,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...,LPFIC[Carbamidomethyl (C)]ELAK,C333,C,333,Carbamidomethyl (C),817.774170,913.765320,742.861450
6538,P00363,GLFAVGECSSVGLHGANR,374,391,602,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...,GLFAVGEC[Carbamidomethyl (C)]SSVGLHGANR,C381,C,381,Carbamidomethyl (C),96.814575,NaN,74.568886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10648,Q59385,SQTIDLTLDGLSCGHCVK,2,19,834,MSQTIDLTLDGLSCGHCVKRVKESLEQRPDVEQADVSITEAHVTGT...,[Acetyl (Protein N-term)]SQTIDLTLDGLSC[Carbami...,C14,C,14,Carbamidomethyl (C),52.428524,333.243897,101.896530
10649,Q59385,SQTIDLTLDGLSCGHCVK,2,19,834,MSQTIDLTLDGLSCGHCVKRVKESLEQRPDVEQADVSITEAHVTGT...,[Acetyl (Protein N-term)]SQTIDLTLDGLSC[Carbami...,C17,C,17,Carbamidomethyl (C),52.428524,333.243897,101.896530
10650,Q7DFV3,GAAVGAGVGAVTEKGR,51,66,114,MKKKILAFGLISALFCSTPAMADMNRTTKGALLGAGVGLLTGNGVN...,GAAVGAGVGAVTEK[NHS-OH]GR,K64,K,64,NHS-OH,32.223633,102.355225,219.477768
10651,P09832,VAIIGAGPAGLACADVLTR,149,167,472,MSQNVYQFIDLQRVDPPKKPLKIRKIEFVEIYEPFSEGQAKAQADR...,VAIIGAGPAGLAC[Carbamidomethyl (C)]ADVLTR,C161,C,161,Carbamidomethyl (C),NaN,38.422024,29.680134


- data is now accessable
- we'll mostly work with entries that have modifications:

# Fetching Proteins

- The setup code also initializes an empty dictionary called proteins
- We can add Proteins to this dictionary to access their info

In [3]:
proteins #no entries yet

{}

- The getProteins() function takes in the whole dataset and goes through each row to add new Proteins to the dictionary
- In theory, it can process every modified protein in the dataset, but for now, it'll go through the first 4 as an example:
- <font color='red'>[This process takes a while. Have not tested if there's a time-out limit]</font> 

In [4]:
modlist = getProteins(moddata)

Querying RCSB Search using the following parameters:
 {"query": {"type": "terminal", "service": "text", "parameters": {"attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession", "operator": "exact_match", "value": "P00350"}}, "request_options": {"return_all_hits": true}, "return_type": "polymer_entity"} 

Querying RCSB Search using the following parameters:
 {"query": {"type": "terminal", "service": "text", "parameters": {"attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession", "operator": "exact_match", "value": "P00363"}}, "request_options": {"return_all_hits": true}, "return_type": "polymer_entity"} 

Querying RCSB Search using the following parameters:
 {"query": {"type": "terminal", "service": "text", "parameters": {"attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession", "operator": "exact_match", "value": "P00509"}}, "request_opti

## <font color='red'>Future Plans:</font> 
This step takes the longest; will look into different methods to speed it up (later, if possible):
- parallel processing
- caching Notebook session 

In [5]:
proteins #4 Proteins have been stored

{'P00350': <protds_v2.Protein at 0x256dddb2550>,
 'P00363': <protds_v2.Protein at 0x256dddb2d30>,
 'P00509': <protds_v2.Protein at 0x256e8890d60>,
 'P00561': <protds_v2.Protein at 0x256ebd40370>}

In [6]:
moddata[:8] #the first 4 proteins that we've looked at correspond to these rows:

,ProteinID,PeptideSequence,Peptide_Start,Peptide_End,Proteinlength,ProteinSequence,ModifiedSequence,ModifiedLocation,ModifiedAA,ModifiedLocationNum,ModifiedType,5min,20min,60min
6534,P00350,VAASKVLSGPQAQPAGDK,297,314,468,MSKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAEN...,VAASK[NHS-OH]VLSGPQAQPAGDK,K301,K,301,NHS-OH,10.097795,42.046806,32.038967
6535,P00350,IAAVAEDGEPCVTYIGADGAGHYVK,159,183,468,MSKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAEN...,IAAVAEDGEPC[Carbamidomethyl (C)]VTYIGADGAGHYVK,C169,C,169,Carbamidomethyl (C),199.737198,317.574097,215.253174
6536,P00363,AAIAAAQANPNAKIALISK,20,38,602,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...,AAIAAAQANPNAK[NHS-OH]IALISK,K32,K,32,NHS-OH,5.593114,4.614741,17.737122
6537,P00363,LPFICELAK,329,337,602,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...,LPFIC[Carbamidomethyl (C)]ELAK,C333,C,333,Carbamidomethyl (C),817.774170,913.765320,742.861450
6538,P00363,GLFAVGECSSVGLHGANR,374,391,602,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...,GLFAVGEC[Carbamidomethyl (C)]SSVGLHGANR,C381,C,381,Carbamidomethyl (C),96.814575,NaN,74.568886
6539,P00509,KAEQYLLENETTK,51,63,396,MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...,K[NHS-OH]AEQYLLENETTK,K51,K,51,NHS-OH,25.769028,42.884392,95.345291
6540,P00509,CTQELLFGK,77,85,396,MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...,C[Carbamidomethyl (C)]TQELLFGK,C77,C,77,Carbamidomethyl (C),27.146547,15.485854,37.134285
6541,P00561,YVGNIDEDGVCR,745,756,820,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,YVGNIDEDGVC[Carbamidomethyl (C)]R,C755,C,755,Carbamidomethyl (C),9.484849,17.113159,43.959064


For convenience reasons, getProteins() also returns these rows as a list:  
(refer to Modified Entries section for details) 

In [7]:
modlist #simplifies them as a list

[['P00350', 301, 'VAASK[NHS-OH]VLSGPQAQPAGDK', 6534],
 ['P00350', 169, 'IAAVAEDGEPC[Carbamidomethyl (C)]VTYIGADGAGHYVK', 6535],
 ['P00363', 32, 'AAIAAAQANPNAK[NHS-OH]IALISK', 6536],
 ['P00363', 333, 'LPFIC[Carbamidomethyl (C)]ELAK', 6537],
 ['P00363', 381, 'GLFAVGEC[Carbamidomethyl (C)]SSVGLHGANR', 6538],
 ['P00509', 51, 'K[NHS-OH]AEQYLLENETTK', 6539],
 ['P00509', 77, 'C[Carbamidomethyl (C)]TQELLFGK', 6540],
 ['P00561', 755, 'YVGNIDEDGVC[Carbamidomethyl (C)]R', 6541]]

## Fetching Individual Proteins

It is possible to add individual proteins through their name with searchPDB():  
- getProteins() from before basically loops through searchPDB() for us
- [Proteins with a lot of PDB results will take longer to complete]

In [8]:
#example for ProteinIDs Q7DFV3 and P0A7K2
searchPDB('Q7DFV3')
searchPDB('P0A7K2') #has 20 results; takes a while

Querying RCSB Search using the following parameters:
 {"query": {"type": "terminal", "service": "text", "parameters": {"attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession", "operator": "exact_match", "value": "Q7DFV3"}}, "request_options": {"return_all_hits": true}, "return_type": "polymer_entity"} 

There were no results for Q7DFV3 

Querying RCSB Search using the following parameters:
 {"query": {"type": "terminal", "service": "text", "parameters": {"attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession", "operator": "exact_match", "value": "P0A7K2"}}, "request_options": {"return_all_hits": true}, "return_type": "polymer_entity"} 



In [9]:
# no results for Q7DFV3; P0A7K2 now added to proteins
proteins

{'P00350': <protds_v2.Protein at 0x256dddb2550>,
 'P00363': <protds_v2.Protein at 0x256dddb2d30>,
 'P00509': <protds_v2.Protein at 0x256e8890d60>,
 'P00561': <protds_v2.Protein at 0x256ebd40370>,
 'P0A7K2': <protds_v2.Protein at 0x256ddd88e20>}

# Accessing Protein Information

After storing a Protein, its information can be accessed with proteins['ProtID']  
Take Protein P00350 for example:

In [10]:
P00350 = proteins['P00350']; P00350

## Some Information available:

In [11]:
# Full list of features from UniProt
P00350.record.features

[FeatureTable(FeatureLocation(ExactPosition(0), ExactPosition(468)), type='CHAIN', id='PRO_0000090037'),
 FeatureTable(FeatureLocation(ExactPosition(9), ExactPosition(15)), type='NP_BIND'),
 FeatureTable(FeatureLocation(ExactPosition(32), ExactPosition(35)), type='NP_BIND'),
 FeatureTable(FeatureLocation(ExactPosition(73), ExactPosition(76)), type='NP_BIND'),
 FeatureTable(FeatureLocation(ExactPosition(127), ExactPosition(130)), type='REGION'),
 FeatureTable(FeatureLocation(ExactPosition(185), ExactPosition(187)), type='REGION'),
 FeatureTable(FeatureLocation(ExactPosition(182), ExactPosition(183)), type='ACT_SITE'),
 FeatureTable(FeatureLocation(ExactPosition(189), ExactPosition(190)), type='ACT_SITE'),
 FeatureTable(FeatureLocation(ExactPosition(101), ExactPosition(102)), type='BINDING'),
 FeatureTable(FeatureLocation(ExactPosition(101), ExactPosition(102)), type='BINDING'),
 FeatureTable(FeatureLocation(ExactPosition(190), ExactPosition(191)), type='BINDING'),
 FeatureTable(FeatureL

In [12]:
#Active/Binding sites:
dict(zip(P00350.getSites()[1],P00350.getSites()[0]))

{'NP_BIND': [74, 75, 76],
 'REGION': [186, 187],
 'ACT_SITE': [190],
 'BINDING': [451]}

In [13]:
#List of PDB results:
P00350.getPDBs()

['2ZYA', '2ZYD', '3FWN']

## Structure Information

Proteins can have multiple structures associated with them. Our example (P00350) has 3:

In [14]:
P00350.structures #each is a PDB object containing the structure's PDBid (name) and the structure data (coordinates)

In [15]:
#take the first of these as an example:
ex = P00350.structures[0]
ex

In [16]:
#name
ex.PDBid

'2ZYA'

In [17]:
#coordinates
ex.structure

array([
	Atom(np.array([40.485, 82.717, 55.529], dtype=float32), chain_id="A", res_id=2, ins_code="", res_name="SER", hetero=False, atom_name="N", element="N"),
	Atom(np.array([40.763, 81.257, 55.403], dtype=float32), chain_id="A", res_id=2, ins_code="", res_name="SER", hetero=False, atom_name="CA", element="C"),
	Atom(np.array([40.344, 80.523, 56.685], dtype=float32), chain_id="A", res_id=2, ins_code="", res_name="SER", hetero=False, atom_name="C", element="C"),
	Atom(np.array([40.704, 80.933, 57.807], dtype=float32), chain_id="A", res_id=2, ins_code="", res_name="SER", hetero=False, atom_name="O", element="O"),
	Atom(np.array([42.253, 81.032, 55.157], dtype=float32), chain_id="A", res_id=2, ins_code="", res_name="SER", hetero=False, atom_name="CB", element="C"),
	Atom(np.array([42.997, 81.467, 56.286], dtype=float32), chain_id="A", res_id=2, ins_code="", res_name="SER", hetero=False, atom_name="OG", element="O"),
	Atom(np.array([39.575, 79.452, 56.525], dtype=float32), chain_id="A", 

The structures are stored as a biotite.structure, so they can be parsed & analyzed using tools from the biotite package  
reference: https://www.biotite-python.org/apidoc/biotite.structure.html#module-biotite.structure  
Examples:

In [18]:
#getting sequence of chain A 
chainSeq('A', ex.structure)

ProteinSequence("SKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAENPGKKLVPYYTVKEFVESLETPRRILLMVKAGAGTDAAIDSLKPYLDKGDIIIDGGNTFFQDTIRRNRELSAEGFNFIGTGVSGGEEGALKGPSIMPGGQKEAYELVAPILTKIAAVAEDGEPCVTYIGADGAGHYVKMVHNGIEYGDMQLIAEAYSLLKGGLNLTNEELAQTFTEWNNGELSSYLIDITKDIFTKKDEDGNYLVDVILDEAANKGTGKWTSQSALDLGEPLSLITESVFARYISSLKDQRVAASKVLSGPQAQPAGDKAEFIEKVRRALYLGKIVSYAQGFSQLRAASEEYNWDLNYGEIAKIFRAGCIIRAQFLQKITDACAENPQIANLLLAPYFKQIADDYQQALRDVVAYAVQNGIPVPTFSAAVAYYDSYRAAVLPANLIQAQRDYFGAHTYKRIDKEGVFHTEW")

In [19]:
#checking to see which chains match the entry's ModifiedSequence:
modSequence = 'VAASK[NHS-OH]VLSGPQAQPAGDK'
checkChains(ex.structure, modSequence)
#(Both chains A and B contain the modSequence)

['A', 'B']

In [20]:
#comparing sequence alignments for chain A and B
import biotite.sequence.align as align
alignment, order, guide_tree, distance_matrix = align.align_multiple(
    [chainSeq('A', ex.structure), chainSeq('B', ex.structure)],
    matrix=align.SubstitutionMatrix.std_protein_matrix(),
    gap_penalty=-5,
    terminal_penalty=False
)
print(alignment)

SKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAENPGKKLVPYYTVKEFVESLETPRRIL
SKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAENPGKKLVPYYTVKEFVESLETPRRIL

LMVKAGAGTDAAIDSLKPYLDKGDIIIDGGNTFFQDTIRRNRELSAEGFNFIGTGVSGGEEGALKGPSIM
LMVKAGAGTDAAIDSLKPYLDKGDIIIDGGNTFFQDTIRRNRELSAEGFNFIGTGVSGGEEGALKGPSIM

PGGQKEAYELVAPILTKIAAVAEDGEPCVTYIGADGAGHYVKMVHNGIEYGDMQLIAEAYSLLKGGLNLT
PGGQKEAYELVAPILTKIAAVAEDGEPCVTYIGADGAGHYVKMVHNGIEYGDMQLIAEAYSLLKGGLNLT

NEELAQTFTEWNNGELSSYLIDITKDIFTKKDEDGNYLVDVILDEAANKGTGKWTSQSALDLGEPLSLIT
NEELAQTFTEWNNGELSSYLIDITKDIFTKKDEDGNYLVDVILDEAANKGTGKWTSQSALDLGEPLSLIT

ESVFARYISSLKDQRVAASKVLSGPQAQPAGDKAEFIEKVRRALYLGKIVSYAQGFSQLRAASEEYNWDL
ESVFARYISSLKDQRVAASKVLSGPQAQPAGDKAEFIEKVRRALYLGKIVSYAQGFSQLRAASEEYNWDL

NYGEIAKIFRAGCIIRAQFLQKITDACAENPQIANLLLAPYFKQIADDYQQALRDVVAYAVQNGIPVPTF
NYGEIAKIFRAGCIIRAQFLQKITDACAENPQIANLLLAPYFKQIADDYQQALRDVVAYAVQNGIPVPTF

SAAVAYYDSYRAAVLPANLIQAQRDYFGAHTYKRIDKEGVFHTEW-
SAAVAYYDSYRAAVLPANLIQAQRDYFGAHTYKRIDKEGVFHTEWL


In [21]:
#center of mass
struc.mass_center(ex.structure)

array([34.18070235, 43.13484544, 90.16028499])

In [22]:
#distance between chain A's and chain B's center of mass
A = ex.structure[ex.structure.chain_id=='A']
B = ex.structure[ex.structure.chain_id=='B']
struc.distance(struc.mass_center(A), struc.mass_center(B))

31.865

# Modified Entries 

Most of our analysis involves looking at each row, so it's nice to have quick access to relevant row info
- in theory, the final program will look at every row
- this example only looked at the first 8 rows (containing the first 4 unique proteins found)

In [21]:
moddata[:8] #these were rows used for this example

,ProteinID,PeptideSequence,Peptide_Start,Peptide_End,Proteinlength,ProteinSequence,ModifiedSequence,ModifiedLocation,ModifiedAA,ModifiedLocationNum,ModifiedType,5min,20min,60min
6534,P00350,VAASKVLSGPQAQPAGDK,297,314,468,MSKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAEN...,VAASK[NHS-OH]VLSGPQAQPAGDK,K301,K,301,NHS-OH,10.097795,42.046806,32.038967
6535,P00350,IAAVAEDGEPCVTYIGADGAGHYVK,159,183,468,MSKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAEN...,IAAVAEDGEPC[Carbamidomethyl (C)]VTYIGADGAGHYVK,C169,C,169,Carbamidomethyl (C),199.737198,317.574097,215.253174
6536,P00363,AAIAAAQANPNAKIALISK,20,38,602,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...,AAIAAAQANPNAK[NHS-OH]IALISK,K32,K,32,NHS-OH,5.593114,4.614741,17.737122
6537,P00363,LPFICELAK,329,337,602,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...,LPFIC[Carbamidomethyl (C)]ELAK,C333,C,333,Carbamidomethyl (C),817.774170,913.765320,742.861450
6538,P00363,GLFAVGECSSVGLHGANR,374,391,602,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...,GLFAVGEC[Carbamidomethyl (C)]SSVGLHGANR,C381,C,381,Carbamidomethyl (C),96.814575,NaN,74.568886
6539,P00509,KAEQYLLENETTK,51,63,396,MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...,K[NHS-OH]AEQYLLENETTK,K51,K,51,NHS-OH,25.769028,42.884392,95.345291
6540,P00509,CTQELLFGK,77,85,396,MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...,C[Carbamidomethyl (C)]TQELLFGK,C77,C,77,Carbamidomethyl (C),27.146547,15.485854,37.134285
6541,P00561,YVGNIDEDGVCR,745,756,820,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,YVGNIDEDGVC[Carbamidomethyl (C)]R,C755,C,755,Carbamidomethyl (C),9.484849,17.113159,43.959064


In [21]:
#the list version of these rows come in handy for easy access:
modlist 

[['P00350', 301, 'VAASK[NHS-OH]VLSGPQAQPAGDK', 6534],
 ['P00350', 169, 'IAAVAEDGEPC[Carbamidomethyl (C)]VTYIGADGAGHYVK', 6535],
 ['P00363', 32, 'AAIAAAQANPNAK[NHS-OH]IALISK', 6536],
 ['P00363', 333, 'LPFIC[Carbamidomethyl (C)]ELAK', 6537],
 ['P00363', 381, 'GLFAVGEC[Carbamidomethyl (C)]SSVGLHGANR', 6538],
 ['P00509', 51, 'K[NHS-OH]AEQYLLENETTK', 6539],
 ['P00509', 77, 'C[Carbamidomethyl (C)]TQELLFGK', 6540],
 ['P00561', 755, 'YVGNIDEDGVC[Carbamidomethyl (C)]R', 6541]]

These "entries" are lists formated:  
[ProteinID, ModifiedLocationNum, ModifiedSequence, index] 

When asking for some information about an entry, the default input is usually in this form. For example:
- entryView([ProteinID, ModifiedLocationNum, ModifiedSequence, index]) for viewing a row's structure + highlighted sites
- entryDist([ProteinID, ModifiedLocationNum, ModifiedSequence, index]) for viewing the distances between the modLocation and sites for a row

This is how the program handles row entries in the background. It refers to this list structure rather than the DataFrame rows themselves  
  
Generally the user isn't involved with these lists directly. However, for testing rows one-by-one, notice that the processRow(rowNum, moddata) will call searchPDB() and return these entries for a single requested row.

In [23]:
#example row 7234:
display(moddata.iloc[700].to_frame().T)
processRow(7234, moddata)

,ProteinID,PeptideSequence,Peptide_Start,Peptide_End,Proteinlength,ProteinSequence,ModifiedSequence,ModifiedLocation,ModifiedAA,ModifiedLocationNum,ModifiedType,5min,20min,60min
7234,P0A7K2,TEFDVILKAAGANK,53,66,121,MSITKDQIIEAVAAMSVMDVVELISAMEEKFGVSAAAAVAVAAGPV...,TEFDVILK[NHS-OH]AAGANK,K60,K,60,NHS-OH,NaN,215.710968,197.620346


['P0A7K2', 60, 'TEFDVILK[NHS-OH]AAGANK', 7234]